## 初始化rds并导入ddl（如果已经做过则跳过）
 * 确保你的aws configure正确设置aksk及region
 * 确保网络在同一VPC且入站规则互联互通

In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.0/291.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 27.9 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.6/158.6 kB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 10.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.3/45.3 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.1/384.1 kB 63.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.8/395.8 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 11.6 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: numexpr
    Found existing installation: numexpr 2.7.3
    Uninstalling numexpr-2.7.

In [ ]:
#!chmod 777 ./setup.sh
#!./setup.sh

In [2]:
import os
import sagemaker
import boto3
import json
from typing import Dict
from typing import Any, Dict, List, Optional

sess = sagemaker.Session()
# sagemaker session bucket -> used for uploading data, models and logs
# sagemaker will automatically create this bucket if it not exists
sagemaker_session_bucket=None
if sagemaker_session_bucket is None and sess is not None:
    # set to default bucket if a bucket name is not given
    sagemaker_session_bucket = sess.default_bucket()

try:
    role = sagemaker.get_execution_role()
except ValueError:
    iam = boto3.client('iam')
    role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']

sess = sagemaker.Session(default_bucket=sagemaker_session_bucket)
sm_client = boto3.client("sagemaker-runtime")

print(f"sagemaker role arn: {role}")
print(f"sagemaker bucket: {sess.default_bucket()}")
print(f"sagemaker session region: {sess.boto_region_name}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
sagemaker role arn: arn:aws:iam::687912291502:role/webui-notebook-stack-ExecutionRole-62U5FV4LJQS
sagemaker bucket: sagemaker-us-west-2-687912291502
sagemaker session region: u

## 初始化langchain环境

In [3]:
from langchain.vectorstores import OpenSearchVectorSearch
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory,ConversationBufferMemory
from langchain import LLMChain
from typing import Any, Dict, List, Union,Mapping, Optional, TypeVar, Union
import os
import time
import json
from opensearchpy import OpenSearch, RequestsHttpConnection
from requests_aws4auth import AWS4Auth

aos_endpoint="vpc-domain66ac69e0-ckglcu7bnpvw-x4tbe22ur5fbqe44iex4phvvwm.us-west-2.es.amazonaws.com"
embedding_endpoint_name="bge-zh-15-2023-09-25-07-02-01-080-endpoint"
region='us-west-2'
username="admin"
passwd="(OL>0p;/"
index_name="chatbot-example-index"
size=10

credentials = boto3.Session().get_credentials()
region = boto3.Session().region_name
awsauth = AWS4Auth(credentials.access_key, credentials.secret_key, region, 'es', session_token=credentials.token)
pwdauth = (username, passwd)


## for embedding
class EmbeddingContentHandler(EmbeddingsContentHandler):
    parameters = {
        "max_new_tokens": 50,
        "temperature": 0,
        "min_length": 10,
        "no_repeat_ngram_size": 2,
    }
    def transform_input(self, inputs: list[str], model_kwargs: Dict) -> bytes:
        input_str = json.dumps({"inputs": inputs, **model_kwargs})
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["sentence_embeddings"]

embedding_content_handler=EmbeddingContentHandler()
    
sm_embeddings = SagemakerEndpointEmbeddings(
    # endpoint_name="endpoint-name", 
    # credentials_profile_name="credentials-profile-name", 
    #endpoint_name="huggingface-textembedding-bloom-7b1-fp1-2023-04-17-03-31-12-148", 
    endpoint_name="bge-zh-15-2023-09-25-07-02-01-080-endpoint",
    region_name="us-west-2", 
    content_handler=embedding_content_handler
)

In [11]:
import boto3
import json
import requests
import time
from collections import defaultdict
from requests_aws4auth import AWS4Auth
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from langchain.vectorstores import OpenSearchVectorSearch
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain
from datetime import datetime




def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "parameters": parameters,
                "is_query" : True,
                "instruction" :  "为这个句子生成表示以用于检索相关文章："
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

def k_nn_ingestion_by_aos(docs,index,hostname,username,passwd):
    current_date = datetime.utcnow()  # 使用 utcnow() 获取当前 UTC 时间
    formatted_date = current_date.strftime('%Y-%m-%dT%H:%M:%SZ')

    auth = (username, passwd)
    search = OpenSearch(
        hosts = [{'host': aos_endpoint, 'port': 443}],
        http_auth = awsauth ,
        #http_auth = auth ,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    for doc in docs:
        intention = doc['intention']
        query = doc['query']
        reply = doc['reply']
        doc_title = doc["doc_title"]
        embedding = doc["embedding"]
        publish_date = formatted_date
        document = { "intention": intention, 'query':query, "reply": reply,"doc_title":doc_title,"embedding":embedding}
        search.index(index=index, body=document)
        

### 意图索引创建及ingestion

In [ ]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
auth = (username, passwd)

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name="chatbot-example-index"
schema={
    "settings" : {
        "index":{
            "number_of_shards" : 1,
            "number_of_replicas" : 0,
            "knn": "true",
            "knn.algo_param.ef_search": 32
        }
    },
    "mappings": {
        "properties": {
            "publish_date" : {
                "type": "date",
                "format": "yyyy-MM-dd HH:mm:ss"
            },
            "intention" : {
                "type" : "keyword"
            },
            "query": {
                "type": "text",
                "analyzer": "ik_max_word",
                "search_analyzer": "ik_smart"
            },
            "reply": {
                "type": "text"
            },
           "doc_title": {
                "type": "keyword"
            },
            "embedding": {
                "type": "knn_vector",
                "dimension": 1024,
                "method": {
                    "name": "hnsw",
                    "space_type": "cosinesimil",
                    "engine": "nmslib",
                    "parameters": {
                        "ef_construction": 512,
                        "m": 32
                    }
                }            
            }
        }
    }
}
search = OpenSearch(
    hosts = [{'host': aos_endpoint, 'port': 443}],
    ##http_auth = awsauth ,
    http_auth = auth ,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)
search.indices.create(index=index_name, body=schema)

In [13]:
intentions = """闲聊
数据分析
知识问答
"""
intention = intentions.split("\n")

querys = """早上好，你好吗?
车牌归属城市为'成都'的车辆累计有多少
什么是波长"""
query=querys.split("\n")

doc_titles = """sample_intentions
sample_intentions
sample_intentions
"""
doc_title=doc_titles.split("\n")

In [14]:
#import sys
#sys.path.append("./code/")
#import func
index_name="chatbot-example-index"
embedding_endpoint_name="bge-zh-15-2023-09-25-07-02-01-080-endpoint"
##########embedding by llm model##############
intention_vectors = []
intention_vectors=get_vector_by_sm_endpoint(query,sm_client,embedding_endpoint_name)

In [15]:
docs=[]
for index, intention_vectors in enumerate(intention_vectors):
    #print(index, sentence_vector)
    #doc = {
    #    "metadata_type":"table",
    #    "database_name":dbs[index],
    #    "table_name": tables[index],
    #    "query_desc_text":querys[index],
    #    "query_desc_embedding": sentence_vector
    #      }
    doc = {
        "intention":intention[index],
        "query":query[index],
        "doc_title": doc_title[index],
        "reply":"",
        "embedding": intention_vectors
          }
    docs.append(doc)

#print((doc["database_name"]))
#########ingestion into aos ###################
k_nn_ingestion_by_aos(docs,index_name,aos_endpoint,username,passwd)

### 测试intentsion调用 #####

In [21]:
from boto3 import client as boto3_client 
lambda_client = boto3_client('lambda')
question = '归属地为昆明的车牌号有多少？'
msg = {
    "fewshot_cnt":5,
    "query": question,
    "use_bedrock" : "True", 
    "embedding_endpoint":embedding_endpoint_name, 
    "region":"us-west-2",
    "aos_endpoint":aos_endpoint,
    "index_name":"chatbot-example-index",
    "llm_model_name":"anthropic.claude-v2"
}



#invoke_response = lambda_client.invoke(
#    FunctionName="Detect_Intention", 
#    InvocationType='RequestResponse',
#    Payload=json.dumps(msg))

In [ ]:
import json
import os
import logging
from collections import Counter

from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain.vectorstores import OpenSearchVectorSearch
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain import PromptTemplate, SagemakerEndpoint
from typing import Any, Dict, List, Union,Mapping, Optional, TypeVar, Union
from langchain.chains import LLMChain
from langchain.llms.bedrock import Bedrock
from botocore.exceptions import ClientError
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth, helpers
import boto3

logger = logging.getLogger()
logger.setLevel(logging.INFO)

credentials = boto3.Session().get_credentials()

class APIException(Exception):
    def __init__(self, message, code: str = None):
        if code:
            super().__init__("[{}] {}".format(code, message))
        else:
            super().__init__(message)


def handle_error(func):
    """Decorator for exception handling"""

    def wrapper(*args, **kwargs):
        try:
            return func(*args, **kwargs)
        except APIException as e:
            logger.exception(e)
            raise e
        except Exception as e:
            logger.exception(e)
            raise RuntimeError(
                "Unknown exception, please check Lambda log for more details"
            )

    return wrapper

class ContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, inputs: List[str], model_kwargs: Dict) -> bytes:
        instruction_zh = "为这个句子生成表示以用于检索相关文章："
        instruction_en = "Represent this sentence for searching relevant passages:"
        input_str = json.dumps({"inputs": inputs, "parameters":{}, "is_query":False, "instruction":instruction_en})
        return input_str.encode('utf-8')

    def transform_output(self, output: bytes) -> List[List[float]]:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["sentence_embeddings"]

class llmContentHandler(LLMContentHandler):
    def transform_input(self, prompt: str, model_kwargs: Dict) -> bytes:
        input_str = json.dumps({'inputs': prompt,'history':[],**model_kwargs})
        return input_str.encode('utf-8')
    
    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["outputs"]

def create_intention_prompt_templete():
    prompt_template = """{instruction}\n\n{fewshot}\n\nHuman: \"{query}\"，这个问题的提问意图是啥？可选项[{options}]\nAssistant: """

    PROMPT = PromptTemplate(
        template=prompt_template, 
        input_variables=['fewshot','query', 'instruction', 'options']
    )
    return PROMPT
    
def get_bedrock_aksk(secret_name='chatbot_bedrock', region_name = "us-west-2"):
    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        # For a list of exceptions thrown, see
        # https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
        raise e

    # Decrypts secret using the associated KMS key.
    secret = json.loads(get_secret_value_response['SecretString'])
    return secret['BEDROCK_ACCESS_KEY'],secret['BEDROCK_SECRET_KEY']
    
def lambda_handler(event):
    
    embedding_endpoint = event['embedding_endpoint']
    region = event['region']
    aos_endpoint = event['aos_endpoint']
    index_name = event['index_name']
    query = event['query']
    fewshot_cnt = event['fewshot_cnt']
    use_bedrock = event['use_bedrock']
    #llm_model_endpoint = event['llm_model_endpoint']
    llm_model_name = event['llm_model_name']
    
    logger.info("embedding_endpoint: {}".format(embedding_endpoint))
    logger.info("region:{}".format(region))
    logger.info("aos_endpoint:{}".format(aos_endpoint))
    logger.info("index_name:{}".format(index_name))
    logger.info("fewshot_cnt:{}".format(fewshot_cnt))
    #logger.info("llm_model_endpoint:{}".format(llm_model_endpoint))

    content_handler = ContentHandler()

    embeddings = SagemakerEndpointEmbeddings(
        endpoint_name=embedding_endpoint,
        region_name=region,
        content_handler=content_handler
    )

    #auth = AWSV4SignerAuth(credentials, region)
    #pwdauth = ("test", passwd) 
    docsearch = OpenSearchVectorSearch(
        index_name=index_name,
        embedding_function=embeddings,
        opensearch_url="https://{}".format(aos_endpoint),
        http_auth = auth,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    
    docs = docsearch.similarity_search_with_score(
        query=query, 
        k = fewshot_cnt,
        space_type="cosinesimil",
        vector_field="embedding",
        text_field="query",
        metadata_field='*'
    )

    docs_simple = [ {"query" : doc[0].page_content, "intention" : doc[0].metadata['intention'], "score":doc[1]} for doc in docs]

    intention_list = [doc['intention'] for doc in docs_simple ]
    intention_counter = Counter(intention_list)
    options = set(intention_list)
    options_str = ", ".join(options)

    instruction = "参考下列Example，回答下列选择题："
    examples = [ "Human: \"{}\"，这个问题的提问意图是啥？可选项[{}]\nAssistant: {}".format(doc['query'], options_str, doc['intention']) for doc in docs_simple ]
    fewshot_str = "{}\n{}\n{}".format("<example>", "\n\n".join(examples), "</example>")
    
    parameters = {
        "temperature": 0.01,
    }

    llm = None
    if not use_bedrock:
        llmcontent_handler = llmContentHandler()
        llm=SagemakerEndpoint(
                endpoint_name=llm_model_endpoint, 
                region_name=region, 
                model_kwargs={'parameters':parameters},
                content_handler=llmcontent_handler
            )
    else:
        boto3_bedrock = boto3.client(
            service_name="bedrock-runtime",
            region_name=region
        )
    
        parameters = {
            "max_tokens_to_sample": 20,
            "stop_sequences": ["\n\n"],
            "temperature":0.01,
            "top_p":1
        }
        
        model_id ="anthropic.claude-instant-v1" if llm_model_name == 'claude-instant' else "anthropic.claude-v2"
        llm = Bedrock(model_id=model_id, client=boto3_bedrock, model_kwargs=parameters)

    prompt_template = create_intention_prompt_templete()
    prompt = prompt_template.format(fewshot=fewshot_str, instruction=instruction, query=query, options=options_str)
    print("prompt=="+prompt)
    if len(options) == 1:
        logger.info("Notice: Only Single latent Intention detected.")
        answer = options.pop()
        log_dict = { "prompt" : prompt, "answer" : answer, "examples": docs_simple }
        log_dict_str = json.dumps(log_dict, ensure_ascii=False)
        logger.info(log_dict_str)
        return answer
        
    llmchain = LLMChain(llm=llm, verbose=False, prompt=prompt_template)
    answer = llmchain.run({'fewshot':fewshot_str, "instruction":instruction, "query":query, "options": options_str})
    answer = answer.strip()

    log_dict = { "prompt" : prompt, "answer" : answer , "examples": docs_simple }
    log_dict_str = json.dumps(log_dict, ensure_ascii=False)
    logger.info(log_dict_str)

    if answer not in options:
        answer = intention_counter.most_common(1)[0]
        for opt in options:
            if opt in answer:
                answer = opt
                break
        
    return answer

lambda_handler(msg)


### 精准prompt ingestion #########

In [ ]:
import boto3
import json
import requests
import time
from collections import defaultdict
from requests_aws4auth import AWS4Auth
import os
from opensearchpy import OpenSearch, RequestsHttpConnection
from langchain.vectorstores import OpenSearchVectorSearch
from langchain import PromptTemplate, SagemakerEndpoint
from langchain.chains.question_answering import load_qa_chain
from langchain.embeddings import SagemakerEndpointEmbeddings
from langchain.llms.sagemaker_endpoint import ContentHandlerBase
from langchain.docstore.document import Document
from langchain.memory import ConversationBufferWindowMemory
from langchain import LLMChain



def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                "parameters": parameters,
                "is_query" : True,
                "instruction" :  "为这个句子生成表示以用于检索相关文章："
            }
        ),
        ContentType="application/json",
    )
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings


def k_nn_ingestion_by_aos(docs,index,hostname,username,passwd):
    auth = (username, passwd)
    search = OpenSearch(
        hosts = [{'host': aos_endpoint, 'port': 443}],
        ##http_auth = awsauth ,
        http_auth = auth ,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    for doc in docs:
        query_desc_embedding = doc['query_desc_embedding']
        database_name = doc['database_name']
        table_name = doc['table_name']
        query_desc_text = doc["query_desc_text"]
        document = { "query_desc_embedding": query_desc_embedding, 'database_name':database_name, "table_name": table_name,"query_desc_text":query_desc_text}
        search.index(index=index, body=document)
        
def k_nn_ingestion_by_aos_v2(docs,index,hostname,username,passwd):
    auth = (username, passwd)
    search = OpenSearch(
        hosts = [{'host': aos_endpoint, 'port': 443}],
        ##http_auth = awsauth ,
        http_auth = auth ,
        use_ssl = True,
        verify_certs = True,
        connection_class = RequestsHttpConnection
    )
    for doc in docs:
        exactly_query_embedding = doc['exactly_query_embedding']
        database_name = doc['database_name']
        table_name = doc['table_name']
        exactly_query_text = doc["exactly_query_text"]
        document = { "exactly_query_embedding": exactly_query_embedding, 'database_name':database_name, "table_name": table_name,"exactly_query_text":exactly_query_text}
        search.index(index=index, body=document)

In [ ]:
## data process
all_querys = """2023年7月派车单数量超过26次的4.2米车辆一共有多少辆
请统计历史累计派车单数量、干线派车单数量、城配派车单数量。历史累计的意思是不限定时间范围
奶茶品牌的站点数量和运输货品数量统计
2022年的运输总量是多少吨？请注意：traff_weight的单位是千克，请把单位转换为吨
车牌为'黑RG6696'的车辆的GPS最近定位上传时间、GPS最近定位省份、GPS最近定位城市、GPS最近定位区县、APP最近定位上传时间、APP最近定位省份、APP最近定位城市、APP最近定位区县。给出sql中字段名不要带上库名
车牌归属城市为'成都'的车辆累计有多少？
取货地城市名称为'北京市'的历史累计不重复的车牌有多少？请注意车牌号有可能有重复
货主-行业列表相似于'西餐连锁'的一共有多少个品牌？多少个客户？
品牌名称为'星巴克'的不重复的站点一共有多少个？
查看租户简称为云南,车辆的车厢长为9.6米和15米的外廓车长、核定载重"""
querys = all_querys.split("\n")

all_tables = """ads_bi_quality_monitor_shipping_detail
dws_ots_waybill_info_da
dws_station_portrait_index_sum_da
dws_ots_waybill_info_da
dws_truck_portrait_index_sum_da
dws_truck_portrait_index_sum_da
dws_ots_waybill_info_da
ads_customer_portrait_index_sum_da
dim_customer_enterprise_station_base_info
dim_pub_truck_tenant,dim_pub_truck_info"""
tables=all_tables.split("\n")

all_dbs = """llm
llm
llm
llm
llm
llm
llm
llm
llm
llm"""
dbs=all_dbs.split("\n")

In [ ]:
#import sys
#sys.path.append("./code/")
#import func
index_name="prompt-optimal-index"
embedding_endpoint_name="bge-zh-15-2023-09-25-07-02-01-080-endpoint"
##########embedding by llm model##############
sentense_vectors = []
sentense_vectors=get_vector_by_sm_endpoint(querys,sm_client,embedding_endpoint_name)

In [ ]:
docs=[]
for index, sentence_vector in enumerate(sentense_vectors):
    #print(index, sentence_vector)
    #doc = {
    #    "metadata_type":"table",
    #    "database_name":dbs[index],
    #    "table_name": tables[index],
    #    "query_desc_text":querys[index],
    #    "query_desc_embedding": sentence_vector
    #      }
    doc = {
        "metadata_type":"table",
        "database_name":dbs[index],
        "table_name": tables[index],
        "exactly_query_text":querys[index],
        "exactly_query_embedding": sentence_vector
          }
    docs.append(doc)

#print((doc["database_name"]))
#########ingestion into aos ###################
k_nn_ingestion_by_aos_v2(docs,index_name,aos_endpoint,username,passwd)